In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os



# Define a more complex CNN model with BatchNorm and Dropout
class EnhancedBrainTumorCNN(nn.Module):
    def __init__(self):
        super(EnhancedBrainTumorCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x



import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# Data augmentation and preprocessing pipeline
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Ensure the image is in RGB format
    transforms.RandomHorizontalFlip(),  # Random horizontal flip for data augmentation
    transforms.RandomRotation(20),  # Random rotation for data augmentation
    transforms.Resize((128, 128)),  # Resize to match model input size
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize as per the model requirements
])

# Dataset class
class BrainTumorDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Load images and labels from the folder structure
        for label in ['yes', 'no']:
            folder_path = os.path.join(image_folder, label)
            for image_name in os.listdir(folder_path):
                self.images.append(os.path.join(folder_path, image_name))
                self.labels.append(1 if label == 'yes' else 0)  # Label 1 for tumor, 0 for no tumor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path)
        
        if self.transform:
            image = self.transform(image)
        
        label = self.labels[idx]
        return image, label

# Paths to the image directories
train_image_folder = 'datasets/brain_tumor_dataset'  # This is where images are stored
valid_image_folder = 'datasets/brain_tumor_dataset'

# Load datasets
train_dataset = BrainTumorDataset(image_folder=train_image_folder, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

valid_dataset = BrainTumorDataset(image_folder=valid_image_folder, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

import torch
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Load model
model = EnhancedBrainTumorCNN()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Scheduler (reduce the learning rate if the validation loss plateaus)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

# Train the model
def train_model(model, train_loader, valid_loader, num_epochs=10):
    best_accuracy = 0
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0
        correct, total = 0, 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        # Calculate average loss and accuracy for this epoch
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total

        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_loss = 0
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(valid_loader)
        val_acc = 100 * val_correct / val_total

        # Print results
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

        # Check if this is the best model based on validation accuracy
        if val_acc > best_accuracy:
            best_accuracy = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
        
        # Step the scheduler
        scheduler.step(val_loss)

    print(f"Best Validation Accuracy: {best_accuracy:.2f}%")
    
# Train the model
train_model(model, train_loader, valid_loader, num_epochs=20)
